In [10]:
import tensorflow as tf
import pathlib
import numpy as np
import onnx
from onnx_tf.backend import prepare

In [11]:
onnx_model = onnx.load("model_v10/model_v10.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("tf_model")  # export the model

INFO:tensorflow:Assets written to: tf_model/assets


INFO:tensorflow:Assets written to: tf_model/assets


In [12]:
repr_data = np.load("/Users/francescore/Desktop/HAEEA/representative_data.npy")

In [13]:
converter = tf.lite.TFLiteConverter.from_saved_model("tf_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]

def representative_data_gen():
    for i in range(repr_data.shape[0]):
        yield [np.expand_dims(repr_data[i], axis=0)]

converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8  # or tf.uint8
converter.inference_output_type = tf.int8  # or tf.uint8
tflite_model = converter.convert()

pathlib.Path("model.tflite").write_bytes(tflite_model)


2023-05-08 12:15:43.304454: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:364] Ignored output_format.
2023-05-08 12:15:43.304469: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:367] Ignored drop_control_dependency.
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


11920